# Librerias

In [ ]:
import pandas as pd
import numpy as np

## Librerias para graficación
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset

## Lectura del dataset

In [ ]:
# Descargar archivos parquet y guardarlos localmente
# !python -m wget "https://huggingface.co/api/datasets/amazon_us_reviews/parquet/Video_Games_v1_00/train/0.parquet" -o 0.parquet
# !python -m wget "https://huggingface.co/api/datasets/amazon_us_reviews/parquet/Video_Games_v1_00/train/1.parquet" -o 1.parquet
# !python -m wget "https://huggingface.co/api/datasets/amazon_us_reviews/parquet/Video_Games_v1_00/train/2.parquet" -o 2.parquet

In [ ]:
df0 = pd.read_parquet("0.parquet")
df1 = pd.read_parquet("1.parquet")
df2 = pd.read_parquet("2.parquet")

In [ ]:
df = pd.concat([df0, df1, df2], axis=0)

In [ ]:
sns.catplot(x='star_rating', kind='count', color='r', data=df)
plt.title('Distribución de Ejemplos')
plt.xlabel('star_rating')
plt.ylabel('Conteo')

## Sample del dataset

In [ ]:
def balancear_dataset_por_rating(data_frame):
    # Obtenemos el recuento de cada valor de "rating"
    conteo_ratings = data_frame['star_rating'].value_counts()

    # Obtenemos el mínimo de ocurrencias de "rating" que queremos mantener
    min_ocurrencias = min(conteo_ratings)

    # Filtramos las filas que tienen "rating" mayor o igual al mínimo especificado
    data_frame_balanceado = data_frame.groupby('star_rating').apply(lambda x: x.sample(min_ocurrencias, random_state=42))

    # Restablecemos el índice del DataFrame resultante y eliminamos el índice antiguo
    data_frame_balanceado.reset_index(drop=True, inplace=True)

    return data_frame_balanceado

In [ ]:
df.shape

In [ ]:
df_balanceado = balancear_dataset_por_rating(df)

In [ ]:
df_train = df_balanceado.copy()
df_train['text'] = df_train['review_headline'] + ' ' + df_train['review_body']

In [ ]:
# Se obtiene una muestra del dataset
df_train = df_train.sample(250000,random_state=42)
df_train.reset_index(drop=True, inplace=True)
df_train = df_train[["text", "star_rating"]]

In [ ]:
sns.catplot(x='star_rating', kind='count', color='r', data=df_train)
plt.title('Distribución de Ejemplos')
plt.xlabel('star_rating')
plt.ylabel('Conteo')

# Modelamiento por Arquitecturas

## BOW

In [ ]:
import re
import nltk
from nltk.corpus import stopwords

stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')

In [ ]:
def processing_text(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()
    # Paso 6: Aplicar stemming. Es una forma de enviar las palabras a una raiz común simplificando de esta manera el vocabulario.
    #         por ejemplo las palabras (absurdo, absurdos) que estan en el review 2895 seran llevados a la raiz común "absurd"
    #         y de esta forma se evita tener dos palabras diferentes con el mismo significado en nuestro vocabulario.
    processed_feature = " ".join([stemmer.stem(i) for i in processed_feature.split()])


    return processed_feature

In [ ]:
# texto_para_procesar y labels respectivamente
texto_para_procesar = df_train['text'].values
labels = df_train['star_rating'].values

# El texto ya procesado de cada ejemplo en nuestro dataset lo almacenaremos en la variable "texto_procesado"
texto_procesado = []
for sentence in range(0, len(texto_para_procesar)):
    procesado = processing_text(texto_para_procesar[sentence])
    texto_procesado.append(procesado)

## Word2Vec

In [ ]:
from gensim.models import KeyedVectors
import gensim

In [ ]:
def processing_text_for_embedding(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()
    return processed_feature

In [ ]:
# El texto ya procesado de cada ejemplo en nuestro dataset lo almacenaremos en la variable "texto_procesado"
embedding = []
for sentence in range(0, len(texto_para_procesar)):
    procesado = processing_text_for_embedding(texto_para_procesar[sentence])
    embedding.append(procesado)

In [ ]:
# Cargar modelo Word2Vec de Google News
word2vec_model_path = 'GoogleNews-vectors-negative300.bin.gz'
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

In [ ]:
# Función para convertir el texto procesado en vectores de Word2Vec promediando los vectores de las palabras presentes
def word2vec_features(texto_procesado, word2vec_model, vector_size=300):
    texto_vectors = []
    for sentence in texto_procesado:
        vectors = []
        for word in sentence:
            if word in word2vec_model:
                vectors.append(word2vec_model[word])
        if vectors:
            texto_vectors.append(np.mean(vectors, axis=0))
        else:
            texto_vectors.append(np.zeros(vector_size))
    return np.array(texto_vectors)

In [ ]:
# Ahora creamos los vectores de características utilizando Word2Vec
texto_embedding = word2vec_features(texto_procesado, word2vec_model)

## Naives Bayes (MultinomialNB)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
def create_model_naives(texto_features):
    # Partición del dataset: Seleccionar 80% para entrenamiento, 20% pruebas.
    X_train, X_test, y_train, y_test = train_test_split(texto_features, labels, test_size=0.2, random_state=0)

    # Modelo: Naive Bayes
    model = MultinomialNB()
    model.fit(X_train, y_train)

    # Reporte de clasificación
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions, digits=4))

    # Matriz de confusión
    cm = confusion_matrix(y_test, predictions,labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot()

    return model

### Representacion 1

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=2500, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_naives(texto_features)

### Representacion 2


In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_naives(texto_features)

### Representacion 3


In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=6500, stop_words=stopwords.words('english'))

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_naives(texto_features)

## Logistic Regresion (SDGClassfier)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
def create_model_sdg(texto_features, eta = 0.1):
    # Partición del dataset: Seleccionar 80% para entrenamiento, 20% pruebas.
    X_train, X_test, y_train, y_test = train_test_split(
        texto_features, labels, test_size=0.2, random_state=0)

    # Modelo: Naive Bayes
    # investicar los parámetros en la documentacion y variar el learning_rate
    model = SGDClassifier(loss='log_loss', learning_rate='constant', eta0=eta)
    model.fit(X_train, y_train)

    # Reporte de clasificación
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions, digits=4))

    # Matriz de confusión
    cm = confusion_matrix(y_test, predictions, labels=model.classes_)
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=model.classes_)
    disp.plot()

    return model

### Representacion 1

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=2500, stop_words=stopwords.words('english'))
# max_features representa el tamaño del vocabulario. Vamos a permitir 2500 palabras.
# stop_words le indicamos las palabras de parada para que las ignore en el vocabulario.

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_sdg(texto_features,eta=0.1)

### Representacion 2

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=4500, stop_words=stopwords.words('english'))
# max_features representa el tamaño del vocabulario. Vamos a permitir 2500 palabras.
# stop_words le indicamos las palabras de parada para que las ignore en el vocabulario.

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_sdg(texto_features,eta=0.1)

### Representacion 3

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=6500, stop_words=stopwords.words('english'))
# max_features representa el tamaño del vocabulario. Vamos a permitir 2500 palabras.
# stop_words le indicamos las palabras de parada para que las ignore en el vocabulario.

# Ahora le solicitamos utilizando nuestro conjunto de datos que construya el vocabulario y tambien transforme nuestro texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_sdg(texto_features,eta=0.1)

## Neuronal Network (Sequencial)

In [ ]:
from keras.models import Sequential

### Representacion 1

### Representacion 2

### Representacion 3